# Projeto 2: Regressão Logística e Redes Neurais

O projeto consiste em explorar técnicas de classificação utilizando o _dataset_ [CINIC-10](https://github.com/BayesWatch/cinic-10), que consiste em um conjunto de 100000 imagens anotadas em 10 classes, que são:

- Airplane
- Automobile
- Bird
- Cat
- Deer
- Dog
- Frog
- Horse
- Ship
- Truck

Para resolver o problema, serão implementadas soluções que utilizam regressão logística multinomial ou rede neural.

In [ ]:
import numpy as np
import itertools as it
import matplotlib.pyplot as plt
from random import seed, shuffle, sample
from datetime import datetime
from copy import deepcopy
from time import time

O _dataset_ a ser utilizado já está separado em conjuntos de treinamento, validação e teste, em arquivos **.npz**. Inicialmente, consideraremos apenas os conjuntos de treinamento e validação. Podemos economizar memória lendo os valores alvo como _int8_.

In [ ]:
train = np.load('Dataset/train.npz')
valid = np.load('Dataset/val.npz')
X, X_v  = train['xs'], valid['xs']
Y, Y_v = train['ys'].astype('int8') , valid['ys'].astype('int8')

# Constantes de classe
classes = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

Com os dados em memória, podemos **normalizá-los** de diversas maneiras. As estatísticas utilizadas para normalização são as mesmas para todos os conjuntos. Esse processo pode consumir bastante memória então, para economia, sempre que possível os tipos serão alterados.

In [ ]:
def get_stats(data, choice=1):
    means,stds,mins,ranges = [],[],[],[]
    
    # Stats for normalization
    if choice == 1 or choice == 3:
        mins = np.apply_along_axis(np.amin, 0, data).astype('int16')
        maxs = np.apply_along_axis(np.amax, 0, data).astype('int16')
        ranges = maxs - mins
    if choice == 2 or choice == 3:
        means = np.apply_along_axis(np.mean, 0, data).astype('float16')
    if choice == 2:
        stds = np.apply_along_axis(np.std, 0, data).astype('float16')
    
    return {'mean':means, 'std':stds, 'mins':mins, 'range':ranges}
    
def normalize_data(data, stats, choice=1):
    ''' Returns the normalized dataset.
    
        Parameters:
            data (array) : numpy array with the dataset.
            stats (array): numpy array with stats given by "get_stats". 0: means. 1: stds. 2:mins. 3:ranges
            choice (int) : integer indicating the transformation to be used.

        Returns:
            data (array list):  transformed data (original data is lost).
    '''

    #### Transforming the dataset ####
    # Min-max normalization
    if choice == 1:
        data = np.apply_along_axis(lambda x: (x - stats['mins'])/stats['range'], 1, data).astype('float32')
            
    # Standardization
    elif choice == 2:
        data = np.apply_along_axis(lambda x: (x - stats['mean'])/stats['std'], 1, data).astype('float32')
            
    # Mean normalization
    elif choice == 3:
        data = np.apply_along_axis(lambda x: (x - stats['mean'])/stats['range'], 1, data).astype('float32')

    return data

def out_layers(array):
    '''Converts values from a list to one hot enconded output layers

        Parameters:
            array (list of ints): Contains the integer ID for each class
    
        Returns:
            np.ndarray : 2D array where each colum is a output layer
    '''
    lower = min(array)
    upper = max(array)
    lines = upper-lower+1
    new_arr = np.zeros((lines,len(array)), dtype=np.int8)

    for j,i in enumerate(map(lambda x : x-lower, array)):
       new_arr[i,j] = 1

    return new_arr

Com os dados normalizados, podemos começar a trabalhar com eles. Primeiramente, foi implementada uma **Regressão Logística Multinomial** para tentar solucionar o problema, utilizando a função _softmax_ para múltiplas classes. 

Foi utilizado **Batch Gradient Descent**, sem _early stopping_ e salvando o melhor conjunto de coeficientes.

Para melhor visualização, os custos em cada conjunto por época foram guardados.

In [ ]:
# Activation function
def softmax(x):
    x -= np.max(x, axis=1, keepdims=True)          # Numeric Stability
    x_exp = np.exp(x)
    return x_exp/x_exp.sum(axis=1, keepdims=True)
    
def prob(X, T):
    return softmax(X.dot(T))

def predict(X, T):
    y_scores = X.dot(T)
    return np.argmax(y_scores, axis=1)

# Cost function
def cost(Y, Y_probs):
    correct_probs = Y_probs[np.arange(Y.size), Y]
    return (-1 * log(correct_probs)).mean()

def cost_derivative(X, Y, Y_probs):
    Y_probs[np.arange(Y.size),Y] -= 1
    Y_probs /= Y.size
    return X.T.dot(Y_probs)

def log(x, bound=1e-16):
    return np.log(np.maximum(x,bound))

# Gradient Descent
def gd_step(X, Y, T, Y_prob, alpha):
    return T - alpha * cost_derivative(X, Y, Y_prob)

def gradient_descent(X, Y, X_v, Y_v, T, alpha=0.01, e_lim=300):
    
    # First losses and scores
    Y_prob = prob(X, T)
    Y_v_prob = prob(X_v, T)
    loss = cost(Y, Y_prob)
    best_loss = cost(Y_v, Y_v_prob)

    # History
    best_T = T.copy()
    history = {'cost': [loss], 'v_cost': [best_loss]}
    
    # Descent
    for i in range(e_lim):
        
        # New theta
        T = gd_step(X, Y, T, Y_prob, alpha)
        
        # New scores and losses
        Y_prob = prob(X, T)
        loss = cost(Y, Y_prob)
        
        # Validation
        Y_v_prob = prob(X_v, T)
        v_loss = cost(Y_v, Y_v_prob)
        
        # Updating best loss
        if v_loss < best_loss:
            best_loss = v_loss
            best_T = T.copy()
        
        # History
        history['cost'].append(loss)
        history['v_cost'].append(v_loss)
        print(f"Epoch {i+1:04d}/{e_lim:04d}", f"loss: {loss:.4f} | val loss: {v_loss:.4f}")
    
    print()
    return best_T, history

Com a regressão pronta, basta iniciar os coeficientes para o treinamento. Para isso, foi utilizada a _Xavier Initialization_.

In [1]:
def init_coefs(features, dim2, rand_seed=None):
    rand = np.random.RandomState(seed=rand_seed)
    return np.sqrt(2/(features + 1)) * rand.randn(features, dim2)

In [ ]:
def run_logistic(X, X_v, Y, Y_v, alpha, n_epochs, n_classes):
    '''
        Runs multinomial logistic regression.
        Uses Xavier Random Initialization of coefficients.
        Gradient descent: softmax function.
    '''
    
    T = init_coefs(X.shape[1], n_classes, 57).astype('float32')

    print("Regression:")
    T, hist = gradient_descent(X, Y, X_v, Y_v, T, alpha, n_epochs)
    return T, hist

Após a regressão, pode-se verificar as curvas de aprendizado de treinamento e de validação por meio dos valores salvos.

In [ ]:
def learning_with_history(history):
    '''
        Plots learning curves from history (dictionary of lists)
    '''
    keys = sorted(history.keys())
    for k in keys:
        plt.plot(history[k])
        
    plt.legend(keys, loc='upper left')
    plt.xlabel('Epoch')
    plt.ylabel('Cost')
    plt.title('Learning Curve')
    plt.show()

Para avaliar o desempenho do modelo, diversas métricas podem ser utilizadas, como:
- Accuracy
- Precision
- Recall
- F1-Score

Porém, para múltiplas classes, apenas _accuracy_ possui definição escalável. As outras métricas estão definidas **por classe**, e a média pode ser utilizada como métrica única se desejado. Um modo fácil de calcular essas métricas é com a **matriz de confusão** da solução.

In [ ]:
# Confusion matrix for the results.
def confusion_matrix(Y, Y_pred, classes):
    conf = np.zeros((classes,classes)).astype('int32')
    for i in range(Y.size):
        conf[Y[i], Y_pred[i]] += 1
    return conf

# Accuracy from confusion matrix. True/total
def accuracy(confusion):
    true_amount = confusion.trace()
    total = confusion.sum()
    return true_amount/total
    
# Precision per class from confusion matrix.
def precision(confusion):
    diag = np.diagonal(confusion)
    return diag/confusion.sum(0)

# Recall per class from confusion matrix.
def recall(confusion):
    diag = np.diagonal(confusion)
    return diag/confusion.sum(1)

# F1 Score per class from precision and recall
def f1_score(prec, rec, bound=1):
    return 2*prec*rec/(prec+rec+bound)
    

# Function to calculate metrics for evaluation
def get_metrics(target, predictions, classes):
    conf = confusion_matrix(target, predictions, classes)
    
    # Metrics
    acc = accuracy(conf)
    prec= precision(conf)
    rec = recall(conf)
    f1 = f1_score(prec, rec)
    avg_acc = (prec + rec)/2
    
    return {'accuracy':acc, 'norm_acc':avg_acc, 'precision':prec, 'recall':rec, 'f1':f1}, conf

A matriz de confusão em si também é interessante de visualizar, por conter muitas informações pertinentes ao desempenho do modelo.

In [ ]:
def plot_confusion_matrix(confusion, classes, model='Neural Network'):
    '''
        Plots an already created confusion matrix for a generic amount of classes.
    '''
    
    fig, ax = plt.subplots(1)

    #Bounding box
    ax.spines['bottom'].set_color('black')
    ax.spines['top'].set_color('black')
    ax.spines['left'].set_color('black')
    ax.spines['right'].set_color('black')

    plt.title('Confusion Matrix for ' + model)

    #Ticks
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thr = confusion.max()/2
    for i, j in it.product(range(confusion.shape[0]), range(confusion.shape[1])):
        plt.text(j, i, confusion[i, j],
            horizontalalignment='center',
            color='white' if confusion[i, j] > thr else 'black')

    plt.grid(False)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

    plt.imshow(confusion, interpolation='nearest', cmap='Blues')
    plt.show()

    return fig, ax

In [ ]:
def test_logistic(X, Y, T, classes):
    '''
        Tests logistic regression with implemented metrics.
        Accuracy, normalized accuracy, precision, recall, f1-score.
    '''
    
    pred = predict(X, T)
    met, conf = get_metrics(Y, pred, len(classes)) 
    plot_confusion_matrix(conf, classes, model = 'Multinomial Logistic Regression')
    
    np.set_printoptions(precision=4)
    print(f'Accuracy: {met["accuracy"]:.4f}')
    print(f'Normalized Accuracy: {met["norm_acc"].mean():.4f}')
    print(f'Precision per class: {met["precision"]} (avg. precision: {met["precision"].mean():.4f})')
    print(f'Recall per class: {met["recall"]} (avg. recall: {met["recall"].mean():.4f})')
    print(f'F1-Score per class: {met["f1"]} (avg. f1-score: {met["f1"].mean():.4f})')
    print()

Enfim, adicionando o _bias_ , coluna adicional no início do conjunto de exemplos, para representar o termo _intercept_ , podemos treinar o modelo.

Após muitos testes, parâmetros bons para a regressão definidos foram _learning rate_ de 0.01 e limite de 300 épocas.

In [ ]:
# Normalization
choice = 2
stats = get_stats(X, choice)
X = normalize_data(X, stats, choice)
print("Training Data Normalized!")
X_v = normalize_data(X_v, stats, choice)
print("Validation Data Normalized!")

Xb = np.insert(X, 0, 1, axis=1)
X_vb = np.insert(X_v, 0, 1, axis=1)
print("Bias Added")

#### MULTINOMIAL LOGISTIC REGRESSION ####
T, history = run_logistic(Xb, X_vb, Y, Y_v, 0.01, 300, len(classes))


In [ ]:
learning_with_history(history)

In [ ]:
# Testing
print("Training Metrics:")
test_logistic(Xb, Y, T, classes)
    
print()
print("Validation Metrics:")
test_logistic(X_vb, Y_v, T, classes)